# 시계열 예측 모델

Sequence (시계열) 데이터 다루기

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

**금융 데이터**를 활용한 시계열 데이터 예측

Finance DataReader를 활용하여 시계열 데이터를 로드하고 금융 데이터를 예측합니다.

## 필요한 모듈 import

In [ ]:
import csv
import tensorflow as tf
import numpy as np
import urllib
import pandas as pd

# module import


## Finace DataReader

[링크](https://github.com/FinanceData/FinanceDataReader)

In [ ]:
# 데이터를 받아오기 위해 finance-datareader 모듈을 설치합니다.
!pip install -U finance-datareader

In [ ]:
import FinanceDataReader as fdr

In [ ]:
# 비트코인 시세를 가져옵니다.
data = fdr.DataReader('BTC/KRW')
data

In [ ]:
data['Close'].plot(figsize=(10, 5))

In [ ]:
# 아래 컬럼의 데이터만 추출하여 최초 최대 정규화를 적용하세요.
# ['Open', 'High', 'Low', 'Close', 'Volume']
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
df_scaled = scaler.fit_transform(data[scale_cols])

df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
df_scaled

## Train Set, Validation Set 생성

Train Set / Validation Set 를 분할 합니다.

- ~최근 500개: Train Set
- 최근 500개~: Valid Set

In [ ]:
train = # 코드를 입력해 주세요
valid = # 코드를 입력해 주세요

In [ ]:
train.shape, valid.shape

## Window Dataset Loader

`window_size`, `batch_size`, `shuffle_buffer`는 자유롭게 설정합니다.

In [ ]:
window_size= # 코드를 입력해 주세요
batch_size = # 코드를 입력해 주세요
shuffle_size = # 코드를 입력해 주세요

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

`train_set`와 `validation_set`를 만듭니다.

In [ ]:
train_set = # 코드를 입력해 주세요

validation_set = # 코드를 입력해 주세요

## 모델 정의 (Sequential)

In [ ]:
model = Sequential([
    # 코드를 입력해 주세요
])

모델의 구조 요약을 확인합니다.

In [ ]:
model.summary()

## 컴파일 (compile)

**Optimizer**의 `learning_rate`을 별도 설정합니다.

In [ ]:
model.compile(# 코드를 입력해 주세요)

## ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

In [ ]:
# 코드를 입력해 주세요


## 학습 (fit)

In [ ]:
model.fit(# 코드를 입력해 주세요)

## 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

In [ ]:
model.# 코드를 입력해 주세요

## 예측

In [ ]:
result = model.predict(validation_set)

In [ ]:
result.shape

## 예측 데이터 시각화

In [ ]:
import matplotlib.pyplot as plt

# 테스트의 실제값과 예측값을 시각화 하세요.
plt.figure(figsize=(12, 9))
plt.plot(valid['Close'].iloc[window_size:].values, label='actual')
plt.plot(result.reshape(-1), label='prediction')
plt.legend()
plt.show()